# ParaView Hello Sphere Test 1
This notebook creates a synthetic data source (a sphere), and creates a polygonal display of it.
Then, it creates a ParaView display widget showing the primary render view.
The notebook further demonstrates how we may use interaction widgets (sliders), to change the resolution of the sphere.

In [1]:
from paraview.simple import *

In [2]:
Version = str(GetParaViewVersion())
info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)

print("ParaView Version ", Version)
print("Vendor:   %s" % info.GetVendor())
print("Version:  %s" % info.GetVersion())
print("Renderer: %s" % info.GetRenderer())

ParaView Version  5.9
Vendor:   NVIDIA Corporation
Version:  4.6.0 NVIDIA 450.51.05
Renderer: Tesla P100-PCIE-16GB/PCIe/SSE2


In [3]:
sphere = Sphere(ThetaResolution=32, PhiResolution=32)

rep = Show(sphere, GetActiveView())
rep.Representation='Surface With Edges'

In [4]:
view = GetRenderView()
from ipyparaview.widgets import PVDisplay
pvdisp = PVDisplay(view)
w = display(pvdisp)

PVDisplay(resolution=(400, 400))

In [ ]:
# Any ParaView object has a list of animatable parameters
sphere.ListProperties()

In [ ]:
# Interact from ipywidgets gives us a simple way to interactively control values
# with a callback function
from ipywidgets import interact, IntSlider


# set the Theta and Phi resolution and trigger a pipeline update
def Sphere_resolution(res):
    sphere.ThetaResolution = sphere.PhiResolution = res
    sphere.UpdatePipeline()
    
i = interact(Sphere_resolution, res=IntSlider(min=3, max=48, step=1, value=12))

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

#SaveImage("/users/jfavre/foo2.png")

In [ ]:
def animate():
  for i in range(48):
    Sphere_resolution(i)
    SaveImage(format("/dev/shm/foo%02d.png" % i))

In [ ]:
animate()

In [ ]:
!rm /scratch/snx3000/jfavre/foo.ogg

In [ ]:
!ffmpeg -i /dev/shm/foo%02d.png /dev/shm/foo.ogg

In [ ]:
from ipywidgets import Video

In [ ]:
video = Video.from_file("/scratch/snx3000/jfavre/foo.ogg")

In [ ]:
video